#  Using Ollama Function Calling

This notebook introduces **function calling with Ollama**, a powerful way to extend large language models (LLMs) by connecting them to external tools, APIs, or custom Python functions.  

Instead of just generating text, the model can decide **when to call a function**, pass the right arguments, and then use the returned result to produce a more accurate and useful response.  

With this workflow, you can:  
-  Enable the model to perform actions beyond text generation  
-  Safely structure inputs and outputs using JSON  
-  Connect to real-world data sources (e.g., weather APIs, databases, calculators)  
-  Build interactive and more intelligent applications  

In this notebook, we will:  
1. Define simple Python functions that the model can call  
2. Configure Ollama to use function calling  
3. Demonstrate how the model reasons about when and how to call functions  

In [3]:
#install the required packages. Make sure you have ollama installed on your system and pull one of the "tools" models: https://ollama.com/download
!pip install ollama
# to run on UBELIX, run the followng command in terminal to use ollama:
#singularity exec --nv /storage/research/dsl_shared/solutions/singularity/ollama.sif ollama serve &  

#then choose which model you want to use
#singularity exec --nv /storage/research/dsl_shared/solutions/singularity/ollama.sif ollama run ollama run gpt-oss:20b

In [26]:
import ollama 
import json
from datetime import datetime

<b> Next we write a function that we want the module to use. Good practice to use doc string and define the parameters well</b>

In [27]:
def calculator(num1: float, num2: float, operator: str) -> float:
    """
    A simple calculator function that performs basic arithmetic operations.

    Parameters
    ----------
    num1 : float
        The first number in the calculation.
    num2 : float
        The second number in the calculation.
    operator : str
        The arithmetic operation to perform. 
        Supported values are:
        - "+" : addition
        - "-" : subtraction
        - "*" : multiplication
        - "/" : division

    Returns
    -------
    float
        The result of the calculation.

    Raises
    ------
    ValueError
        If the operator is not supported or if division by zero is attempted.
    """
    if operator == "+":
        return num1 + num2
    elif operator == "-":
        return num1 - num2
    elif operator == "*":
        return num1 * num2
    elif operator == "/":
        if num2 == 0:
            raise ValueError("Division by zero is not allowed.")
        return num1 / num2
    else:
        raise ValueError(f"Unsupported operator: {operator}. Use one of '+', '-', '*', '/'.")



# add new functions?
def read_file(file_path:str):
    pass


def get_datetime():
    pass

In [28]:
# let's see which models are available for us locally. Check on the ollama website which models support tools (function calling)
!ollama list

NAME                ID              SIZE      MODIFIED          
qwen3:latest        500a1f067a9f    5.2 GB    About an hour ago    
qwen3:8b            500a1f067a9f    5.2 GB    2 days ago           
gemma3n:e2b         719372f8c7de    5.6 GB    8 weeks ago          
deepseek-r1:1.5b    e0979632db5a    1.1 GB    3 months ago         


In [29]:
def run_conversation(user_input: str):
    system_msg = {
        "role": "system",
        "content": (
            "You have access to tools. When a tool result is provided, "
            "use it directly to answer the user’s request. "
            "For numbers, state the number clearly. "
            "For text, summarize, explain, or analyze it as needed. "
            "Do not reveal chain-of-thought."
        )
    }

    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculator",
                "description": "A simple calculator for basic arithmetic operations.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "num1": {"type": "number", "description": "First number"},
                        "num2": {"type": "number", "description": "Second number"},
                        "operator": {
                            "type": "string",
                            "enum": ["+", "-", "*", "/"],
                            "description": "Arithmetic operator"
                        },
                    },
                    "required": ["num1", "num2", "operator"],
                },
            },
        },
    ]

    messages = [
        system_msg,
        {"role": "user", "content": user_input},
    ]

    # First turn: let the model decide to call the tool
    response = ollama.chat(
        model="qwen3:8b",
        messages=messages,
        tools=tools,
        options={"temperature": 0}
    )

    tool_calls = response["message"].get("tool_calls", [])
    if not tool_calls:
        print("Model Answer:", response["message"]["content"])
        return

    # Execute tool(s)
    messages.append(response["message"])
    for call in tool_calls:
        fn = call["function"]["name"]
        args = call["function"]["arguments"]

        if fn == "calculator":
            result = calculator(**args)
        else:
            result = f"Unknown tool: {fn}"

        # Wrap result into structured payload
        tool_payload = {
            "tool": fn,
            "inputs": args,
            "result": result
        }

        messages.append({
            "role": "tool",
            "name": fn,
            "content": json.dumps(tool_payload)
        })

    # Second turn: let model generate final answer naturally
    final = ollama.chat(
        model="qwen3:8b",
        messages=messages,
        options={"temperature": 0}
    )

    print("Final Model Answer:", final["message"]["content"])


In [30]:
while True:
        user = input("############ask a question:############### ")
        if user == "exit":
            break
        else:
            run_conversation(user)

############ask a question:###############  can you tell me what is this book about? here is the path: /home/ahmad-unibe/embedding_search/data/short.txt


Final Model Answer: <think>
</think>

The text you provided is a collection of two historical speeches: 

1. **Kennedy's Inaugural Address (January 20, 1961)**: This speech emphasizes themes of freedom, unity, and the responsibility of the nation to promote peace and prosperity. Kennedy calls for global cooperation, the pursuit of justice, and a commitment to the ideals of liberty and democracy.

2. **Lincoln's Gettysburg Address (November 19, 1863)**: Delivered during the American Civil War, this speech reflects on the purpose of the war and honors those who died in battle. Lincoln emphasizes the importance of preserving the Union, the idea of equality, and the enduring nature of a government "of the people, by the people, for the people."

Both speeches are significant historical documents that highlight the values and aspirations of their respective times.


############ask a question:###############  what time is it now


Final Model Answer: <think>
</think>

The current time is 14:01:21.


############ask a question:###############  what is the date?


Final Model Answer: <think>
</think>

The current date and time is September 18, 2025, at 14:01:29.


############ask a question:###############  what is 0.873632 * 2.37892?


Final Model Answer: <think>
</think>

The result of $0.873632 \times 2.37892$ is $2.07830063744$.


############ask a question:###############  exit


In [7]:
calculation_without_tool = """"
>>> what is 0.873632 * 2.37892
Thinking...
Okay, let's see. I need to calculate 0.873632 multiplied by 2.37892 and then divide by...
wait, actually, the original question is 0.873632 * 2.37892 divided by something? Wait, no,
the user wrote "what is 0.873632 * 2.37892 /think". Wait, maybe there's a typo? Because the
original question is written as "0.873632 * 2.37892 /think". Hmm. Wait, maybe the user
intended to write 0.873632 multiplied by 2.37892, and then divided by something else? But the
way it's written here, maybe it's just 0.873632 multiplied by 2.37892, and then divided by...
nothing? Wait, the original question is written as "what is 0.873632 * 2.37892 /think". Maybe
there's a formatting error here. Let me check again.

Wait, perhaps the user meant to write "0.873632 * 2.37892" and then divided by something else,
but the slash is part of the thinking process? Wait, no, the user wrote "what is 0.873632 *
2.37892 /think". Maybe the "/think" is part of the query? Wait, maybe the user is using a
system where they have to think first, so the "/think" is part of the instruction. But in that
case, the actual question is just 0.873632 multiplied by 2.37892. Let me confirm. The original
question is "what is 0.873632 * 2.37892 /think". Maybe the "/think" is a mistake, or maybe
it's part of the problem. But given that the user is asking for the calculation of 0.873632
multiplied by 2.37892, I think that's the main question. Let me proceed with that.

So, first, I need to multiply 0.873632 by 2.37892. Let me write that down. Let me recall how
to multiply decimals. Alternatively, I can convert them to fractions, but that might be
complicated. Alternatively, I can use a calculator, but since I need to do it manually, let me
try to compute it step by step.

First, let me write both numbers:

0.873632

times

2.37892

Let me note that multiplying two decimals can be done by first multiplying the numbers as if
they were whole numbers and then adjusting the decimal places accordingly. However, since both
numbers have decimal points, I need to consider the total number of decimal places.

Alternatively, I can use the standard multiplication method for decimals. Let me try that.

First, let me ignore the decimal points and multiply 873632 by 237892. Then, adjust the
decimal places accordingly.

But wait, that might be a lot of digits. Let me check:

0.873632 has 6 decimal places.

2.37892 has 5 decimal places.

Total decimal places: 6 + 5 = 11. So, the product will have 11 decimal places.

But multiplying 873632 by 237892 manually would be tedious. Maybe there's a better way.

Alternatively, I can use approximation or break down the numbers.

Alternatively, use the distributive property.

Let me think. Let me write 2.37892 as 2 + 0.3 + 0.07 + 0.008 + 0.0009 + 0.00002.

But maybe that's too fragmented. Alternatively, consider 2.37892 as 2.3 + 0.07892.

Alternatively, use the standard multiplication method.

Let me try multiplying 0.873632 by 2.37892.

First, let me write them aligned:

       0.873632
     × 2.37892
     __________

But to make it easier, I can convert them to fractions. However, that might not be
straightforward. Alternatively, multiply them as is.

Let me recall that multiplying decimals can be done by multiplying the numbers as if they were
integers and then placing the decimal point in the result.

So, first, multiply 873632 by 237892. But that's going to be a huge number. Let me see if
there's a better approach.

Alternatively, use estimation to check the magnitude. Let me approximate the numbers:

0.873632 is approximately 0.8736

2.37892 is approximately 2.3789

Multiplying 0.8736 * 2.3789. Let me compute that.

First, 0.8 * 2.3789 = 1.90312

Then, 0.07 * 2.3789 = 0.166523

Then, 0.0036 * 2.3789 ≈ 0.008564

Adding those up: 1.90312 + 0.166523 = 2.069643 + 0.008564 ≈ 2.078207

But this is an approximation. However, the exact value would require more precise calculation.

Alternatively, use a calculator-like approach. Wait, since I need to do this manually, maybe I
can use the standard multiplication method.

Let me write 0.873632 × 2.37892.

First, let me write both numbers with their decimal points:

0.873632

× 2.37892

------------

To perform the multiplication, I can ignore the decimals and multiply 873632 × 237892, then
adjust the decimal places.

But this is going to be a very large multiplication. Let me see if there's another way.

Alternatively, note that 0.873632 × 2.37892 can be written as (8.73632 × 10^-1) × (2.37892 ×
10^0) = 8.73632 × 2.37892 × 10^-1.

So, if I can compute 8.73632 × 2.37892, then divide by 10.

But maybe that's not easier. Let me try multiplying 8.73632 × 2.37892.

Alternatively, use the standard multiplication:

First, multiply 8.73632 by 2.37892.

Let me write it out step by step.

First, write 8.73632 × 2.37892.

Let me convert them to whole numbers by multiplying by 10^6 and 10^5 respectively? Wait, maybe
not. Alternatively, use the standard multiplication steps.

Let me consider that multiplying 8.73632 by 2.37892 is the same as multiplying 8.73632 × 2 +
8.73632 × 0.3 + 8.73632 × 0.07 + 8.73632 × 0.008 + 8.73632 × 0.0009 + 8.73632 × 0.00002.

Let me compute each term:

First term: 8.73632 × 2 = 17.47264

Second term: 8.73632 × 0.3 = 2.620896

Third term: 8.73632 × 0.07 = 0.6115424

Fourth term: 8.73632 × 0.008 = 0.06989056

Fifth term: 8.73632 × 0.0009 = 0.007862688

Sixth term: 8.73632 × 0.00002 = 0.0001747264

Now, add all these terms together:

Start with 17.47264 + 2.620896 = 20.093536

Then add 0.6115424: 20.093536 + 0.6115424 = 20.7050784

Next, add 0.06989056: 20.7050784 + 0.06989056 = 20.77496896

Then add 0.007862688: 20.77496896 + 0.007862688 ≈ 20.782831648

Finally, add 0.0001747264: 20.782831648 + 0.0001747264 ≈ 20.7830063744

So, the total is approximately 20.7830063744.

But remember, this was the multiplication of 8.73632 × 2.37892, and then we need to divide by
10 because the original numbers were 0.873632 (which is 8.73632 × 10^-1) and 2.37892 (which is
2.37892 × 10^0). Therefore, the total product is 20.7830063744 × 10^-1 = 2.07830063744.

Wait, but let me check my steps again to make sure I didn't make a mistake in the calculation.

Starting with the first term: 8.73632 × 2 = 17.47264. Correct.

Second term: 8.73632 × 0.3. 8 × 0.3 = 2.4, 0.73632 × 0.3 = 0.220896, total 2.4 + 0.220896 =
2.620896. Correct.

Third term: 8.73632 × 0.07. Let's compute 8 × 0.07 = 0.56, 0.73632 × 0.07 = 0.0515424. Total
0.56 + 0.0515424 = 0.6115424. Correct.

Fourth term: 8.73632 × 0.008. 8 × 0.008 = 0.064, 0.73632 × 0.008 = 0.00589056. Total 0.064 +
0.00589056 = 0.06989056. Correct.

Fifth term: 8.73632 × 0.0009. 8 × 0.0009 = 0.0072, 0.73632 × 0.0009 = 0.000662688. Total
0.0072 + 0.000662688 = 0.007862688. Correct.

Sixth term: 8.73632 × 0.00002. 8 × 0.00002 = 0.00016, 0.73632 × 0.00002 = 0.0000147264. Total
0.00016 + 0.0000147264 = 0.0001747264. Correct.

Adding them all together:

Start with 17.47264 + 2.620896 = 20.093536. Then +0.6115424 = 20.7050784. Then +0.06989056 =
20.77496896. Then +0.007862688 = 20.782831648. Then +0.0001747264 = 20.7830063744. Yes, that's
correct.

So, dividing by 10 gives 2.07830063744. So approximately 2.07830063744.

But let me check if this is accurate. Alternatively, maybe I can use another method to verify.

Alternatively, use the standard multiplication of decimals:

Multiply 0.873632 × 2.37892.

Let me write them as:

       0.873632
     × 2.37892
     __________

To perform this multiplication, I can consider multiplying 0.873632 by each digit of 2.37892,
considering the decimal places.

But this is going to be complex, so maybe I can use another approach.

Alternatively, use the fact that 0.873632 × 2.37892 = (0.8 + 0.07 + 0.003 + 0.0006 + 0.00003 +
0.000002) × 2.37892.

But this might not be helpful. Alternatively, use the decimal multiplication step-by-step.

Alternatively, use the approximate value I calculated earlier, which is approximately 2.0783.
Let me check with another method.

Let me use the calculator-like approach by breaking down the numbers.

First, note that 0.873632 × 2.37892.

Let me write this as (0.8 + 0.07 + 0.003 + 0.0006 + 0.00003 + 0.000002) × 2.37892.

But maybe that's too detailed. Alternatively, use the standard multiplication:

Let me convert both numbers to fractions. However, this might not be practical.

Alternatively, use the fact that 0.873632 × 2.37892 is equal to (873632 × 10^-6) × (237892 × 10^-5) = 873632 × 237892 × 10^-11.

But calculating 873632 × 237892 is a huge number. Maybe there's a better way.

Alternatively, use an approximate value. Let me check using a calculator (pretending I have one).

Wait, but since I need to do it manually, let me try another approach.

Let me consider that 0.873632 × 2.37892 = ?

Let me approximate 0.873632 as 0.8736 and 2.37892 as 2.3789.

Now, multiplying 0.8736 × 2.3789.

Let me use the standard multiplication:

First, multiply 0.8736 by 2.3789.

Let me write it as:

       0.8736
     × 2.3789
     __________

First, multiply 0.8736 by 2.3789.

Let me do this step by step.

First, multiply 0.8736 by 2 = 1.7472

Then, 0.8736 × 0.3 = 0.26208

Then, 0.8736 × 0.07 = 0.061152

Then, 0.8736 × 0.008 = 0.0069888

Then, 0.8736 × 0.0009 = 0.00078624

Adding all these:

Start with 1.7472 + 0.26208 = 2.00928

Add 0.061152: 2.00928 + 0.061152 = 2.070432

Add 0.0069888: 2.070432 + 0.0069888 = 2.0774208

Add 0.00078624: 2.0774208 + 0.00078624 = 2.07820704

So, approximately 2.07820704.

But earlier, when I did the breakdown with more precise numbers, I got approximately 2.07830063744. The difference is due to the approximation of
rounding 0.873632 to 0.8736 and 2.37892 to 2.3789. So, the exact value is slightly higher.

Therefore, my initial calculation of approximately 2.0783 seems reasonable.

To get more precise, let's consider the exact numbers:

Original numbers: 0.873632 × 2.37892

Let me write them with more decimal places for precision:

Let me use the exact multiplication steps.

Let me consider multiplying 0.873632 by 2.37892.

First, express both numbers as decimals:

0.873632 = 0.873632

2.37892 = 2.37892

Now, multiplying these two numbers.

Let me use the standard multiplication method, which involves multiplying each digit and adding the results, considering the decimal places.

But this is quite tedious manually. Alternatively, I can use the approximate value I calculated earlier, which is around 2.0783.

Alternatively, I can use the fact that 0.873632 × 2.37892 ≈ 2.0783.

To check, let me compute 0.873632 × 2.37892 using another approach.

Let me note that 2.37892 is approximately 2.3789, and 0.873632 is approximately 0.873632.

Let me compute 0.873632 × 2.37892:

First, compute 0.873632 × 2 = 1.747264

Then, compute 0.873632 × 0.3 = 0.2620896

Then, compute 0.873632 × 0.07 = 0.06115424

Then, compute 0.873632 × 0.008 = 0.006989056

Then, compute 0.873632 × 0.0009 = 0.0007862688

Then, compute 0.873632 × 0.00002 = 0.00001747264

Now, adding all these components:

Start with 1.747264

Add 0.2620896: 1.747264 + 0.2620896 = 2.0093536

Add 0.06115424: 2.0093536 + 0.06115424 = 2.07050784

Add 0.006989056: 2.07050784 + 0.006989056 = 2.077496896

Add 0.0007862688: 2.077496896 + 0.0007862688 = 2.0782831648

Add 0.00001747264: 2.0782831648 + 0.00001747264 = 2.07830063744

So, the total is 2.07830063744, which matches my earlier result.

Therefore, the exact value is approximately 2.07830063744.

So, rounding to, say, 8 decimal places, it would be 2.07830064.

But let me confirm if there are any rounding errors in my steps. Since I broke down the multiplication into components, and each step seems accurate,
I think this is correct.

Therefore, the product of 0.873632 and 2.37892 is approximately 2.07830063744.

Now, the original question was to compute this product. Depending on the required precision, the answer can be presented accordingly. If it's a
mathematical problem expecting an exact fraction, we might need to compute it as such, but since these are decimal numbers, it's likely expected to
present the decimal value.

Alternatively, perhaps the question is expecting a simpler approach or there's a trick. Let me check if the numbers are related or if there's a
calculation shortcut.

Looking at the numbers:

0.873632 and 2.37892.

Alternatively, note that 0.873632 × 2.37892 = ?

But without more context, I think my previous calculation is the way to go.

Therefore, the final answer is approximately 2.0783. Depending on the required decimal places, it might be 2.0783 or more precise.

But let me check using another method. Let me use the approximate value of 0.873632 × 2.37892.

Let me use the approximation of multiplying 0.8736 × 2.3789, which I calculated as approximately 2.07820704, and then adjust for the extra decimals in
the original numbers.

But since I already did the precise breakdown, I think the value is accurate.

Therefore, the answer is approximately 2.0783.

But to ensure, let me check with another method. Let me use the formula for multiplication:

Let me consider that 0.873632 × 2.37892 = (873632 × 10^-6) × (237892 × 10^-5) = 873632 × 237892 × 10^-11.

But calculating 873632 × 237892 is a huge number. Let me try to compute it step by step.

But this seems impractical manually. Alternatively, I can use an approximate value.

Alternatively, use the fact that 0.873632 × 2.37892 ≈ 2.0783.

Therefore, unless there's a specific requirement for more decimal places or a fractional form, this should be sufficient.

Thus, the final answer is approximately 2.0783. However, to provide more precision, let's carry out more decimal places.

From the earlier precise calculation, it was 2.07830063744. So, if we round to 8 decimal places, it's 2.07830064.

But depending on the context, it might be presented as 2.0783.

However, since the original numbers have 6 and 5 decimal places respectively, the result should have up to 11 decimal places, but typically, we round
to a reasonable number.

Alternatively, if the question expects an exact fraction, let's try to compute it.

First, note that 0.873632 is equal to 873632/1000000, and 2.37892 is equal to 237892/100000.

So, multiplying them:

(873632 × 237892) / (1000000 × 100000) = (873632 × 237892) / 100000000000.

But calculating the numerator:

873632 × 237892. This is a massive number. Let me try to compute it step by step.

But this is impractical manually. Therefore, it's better to stick with the decimal approximation.

Thus, the final answer is approximately 2.0783.

But to verify once more, let me use another approach: using the fact that 0.873632 × 2.37892 = ?

Let me approximate 0.873632 ≈ 0.8736 and 2.37892 ≈ 2.3789.

Then, 0.8736 × 2.3789:

Let me compute this using the standard multiplication:

First, multiply 0.8736 by 2.3789.

Let me write it as:

0.8736 × 2.3789

Break it down:

First, multiply 0.8736 × 2 = 1.7472

Then, 0.8736 × 0.3 = 0.26208

Then, 0.8736 × 0.07 = 0.061152

Then, 0.8736 × 0.008 = 0.0069888

Then, 0.8736 × 0.0009 = 0.00078624

Adding all these:

1.7472 + 0.26208 = 2.00928

2.00928 + 0.061152 = 2.070432

2.070432 + 0.0069888 = 2.0774208

2.0774208 + 0.00078624 = 2.07820704

Which matches the earlier result of 2.07820704. So, with the approximated numbers, it's 2.07820704, and with the precise numbers, it's 2.07830063744.

Therefore, the precise value is slightly higher due to the extra decimals in the original numbers.

Thus, the final answer is approximately 2.0783.

But to ensure, let me check with another method: using a calculator-like approach.

Assuming I have access to a calculator, I would input 0.873632 × 2.37892.

Let me simulate this:

First, multiply 0.873632 × 2 = 1.747264

Then, multiply 0.873632 × 0.3 = 0.2620896

Then, multiply 0.873632 × 0.07 = 0.06115424

Then, multiply 0.873632 × 0.008 = 0.006989056

Then, multiply 0.873632 × 0.0009 = 0.0007862688

Then, multiply 0.873632 × 0.00002 = 0.00001747264

Now, sum all these:

Start with 1.747264

Add 0.2620896: 1.747264 + 0.2620896 = 2.0093536

Add 0.06115424: 2.0093536 + 0.06115424 = 2.07050784

Add 0.006989056: 2.07050784 + 0.006989056 = 2.077496896

Add 0.0007862688: 2.077496896 + 0.0007862688 = 2.0782831648

Add 0.00001747264: 2.0782831648 + 0.00001747264 = 2.07830063744

Yes, this matches the previous result. Therefore, the exact value is 2.07830063744.

So, depending on the required precision, the answer can be presented as 2.0783 or more precisely. If the question requires rounding to a certain
decimal place, it would be specified, but since it's not mentioned, providing the exact value up to several decimals is appropriate.

Therefore, the final answer is approximately 2.0783.

But to ensure, let me check if there's any possible error in my calculations. All steps seem consistent, and breaking down the multiplication into
components and adding them up leads to the same result. Therefore, I can be confident in this answer.

**Final Answer**
The product of 0.873632 and 2.37892 is \boxed{2.0783}.
...done thinking.

To compute the product of $ 0.873632 \times 2.37892 $, we can use a systematic and precise breakdown of the multiplication process.

---

### Step-by-Step Breakdown:

We start by expressing the two numbers in terms of their components:

$$
0.873632 = 0.8 + 0.07 + 0.003 + 0.0006 + 0.00003 + 0.000002
$$
$$
2.37892 = 2 + 0.3 + 0.07 + 0.008 + 0.0009 + 0.00002
$$

We then compute the product by multiplying each digit pair and summing up the results, handling the decimal places appropriately. This results in a
total that includes the sum of all individual products of the digits.

---

### Precise Calculation:

Each digit multiplication is performed and then added together. The result of this detailed computation (as shown in the thought process) is:

$$
0.873632 \times 2.37892 = 2.07830063744
$$

This is a very precise value. However, in many practical applications, rounding to a reasonable number of decimal places is preferred.

---

### Final Answer:

Depending on the required precision, the product can be rounded to:

$$
\boxed{2.0783}
$$

This is a commonly accepted level of precision for this type of multiplication.
"""

len(calculation_without_tool.split())


date_example = """############ask a question:###############  what is the date and time today?
Model Answer: <think>
</think>

I cannot provide the current date and time as it is not supported by the available tools. Please check your device's clock for the current date and time."""

2916